In [1]:
from rdflib import Namespace, Graph

from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model
from buildingmotif.namespaces import bind_prefixes

In [2]:
# setup our buildingmotif instance
bm = BuildingMOTIF("sqlite://", shacl_engine="topquadrant")

# create the model w/ a namespace
BLDG = Namespace("urn:ex/")
bldg = Model.create(BLDG)
bind_prefixes(bldg.graph)
bldg.graph.bind("bldg", BLDG)

things = []

In [3]:
# load libraries
# NREL templates for 223P
nrel_lib = Library.load(directory="../libraries/ashrae/223p/nrel-templates")
# 223P ontology
s223 = Library.load(ontology_graph="../libraries/ashrae/223p/ontology/223p.ttl")

2024-07-02 17:28:37,453 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/extensions/settings (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:28:37,454 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/model/core (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:28:37,455 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/model/equipment (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:28:37,456 | root |  WARNING: An ontology could not resolve a dependency on http://data.ashrae.org/standard223/1.0/vocab/enumeration (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:28:37,456 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/constant (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:28:37,457 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/currency (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:28:37,457 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/unit (No row was found when one was required). Check this is loaded into BuildingMOTIF


2024-07-02 17:28:37,458 | root |  WARNING: An ontology could not resolve a dependency on http://www.w3.org/ns/shacl# (No row was found when one was required). Check this is loaded into BuildingMOTIF


In [4]:
# get some templates from the library
mau_templ = nrel_lib.get_template_by_name("makeup-air-unit").inline_dependencies()
vav_templ = nrel_lib.get_template_by_name("vav-reheat").inline_dependencies()
duct_templ = nrel_lib.get_template_by_name("duct").inline_dependencies()
zone_templ = nrel_lib.get_template_by_name("hvac-zone").inline_dependencies()
space_templ = nrel_lib.get_template_by_name("hvac-space").inline_dependencies()
fcu_templ = nrel_lib.get_template_by_name("fcu").inline_dependencies()
chws_templ = nrel_lib.get_template_by_name("chilled-water-system").inline_dependencies()
hws_templ = nrel_lib.get_template_by_name("hot-water-system").inline_dependencies()

In [5]:
# create a makeup air unit
mau = mau_templ.evaluate({"name": BLDG.MAU, "air-supply": BLDG.MAU_Supply})
things.append(mau)

/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "cooling-coil-pump-out, supply-fan-oa-flow-switch, heating-coil-return-water-temp, heating-coil-air-out, evaporative-cooler-in-mapsto, cooling-coil, oad-command, cooling-coil-return-water-temp, cooling-coil-pump-vfd-fb, evaporative-cooler-evap-cool-pump-2stage-in, cooling-coil-pump-out-mapsto, evaporative-cooler-evap-cool-pump-2stage-vfd-frq, supply-fan-in, c5, MAU-HRC-air-in-mapsto, heating-coil-air-out-mapsto, supply-fan-vfd-pwr, MAU-HRC-water-in, cooling-coil-pump-onoff-cmd, supply-fan-vfd-energy, evaporative-cooler-evap-cool-pump-2stage-vfd-fb, heating-coil-valve, outside-air-mapsto, evaporative-cooler-evap-cool-pump-2stage-out-mapsto, oad-feedback, MAU-HRC-supply-water-temp, evaporative-cooler-evap-cool-pump-2stage-vfd-pwr, heating-coil-air-in-mapsto, pre-filter-out, MAU-HRC-water-in-mapsto, pre-filter-out-mapsto, heating-coil-water-in, evaporative-cooler-evap-cool-fill-valve-comm

In [6]:
# make 2 VAVs, connect them to the MAU via ducts
vav1 = vav_templ.evaluate({"name": BLDG["VAV-1"], "air-in": BLDG["VAV-1-in"]})
mau_to_vav1 = duct_templ.evaluate({"a": BLDG.MAU_Supply, "b": BLDG["VAV-1-in"]})
things.append(vav1)
things.append(mau_to_vav1)

vav2 = vav_templ.evaluate({"name": BLDG["VAV-2"], "air-in": BLDG["VAV-2-in"]})
things.append(vav2)
mau_to_vav2 = duct_templ.evaluate({"a": BLDG.MAU_Supply, "b": BLDG["VAV-2-in"]})
things.append(mau_to_vav2)

/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "rhc-valve-command, rhc-supply-water-temp, rhc-valve-out-mapsto, air-out-mapsto, sup-air-temp, air-out, rhc-valve-out, rhc-air-in, sup-air-pressure-sensor, c0, rhc-air-out, sup-air-pressure, sup-air-flow-sensor, rhc-water-out-mapsto, rhc-air-in-mapsto, sup-air-flow, dmp-in, dmp-feedback, rhc-supply-water-temp-sensor, dmp, rhc-valve-in, rhc, dmp-command, air-in-mapsto, rhc-valve-feedback, rhc-water-in, rhc-valve-in-mapsto, rhc-valve, dmp-in-mapsto, rhc-water-out, dmp-out, dmp-out-mapsto, sup-air-temp-sensor, rhc-air-out-mapsto, rhc-return-water-temp-sensor, rhc-return-water-temp, rhc-water-in-mapsto" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "name" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412

In [7]:
# VAV1 goes to Zone 1
zone1 = zone_templ.evaluate({"name": BLDG["zone1"]})
things.append(zone1)
zone1space1 = space_templ.evaluate({"name": BLDG["zone1space1"], "zone": BLDG["zone1"]})
things.append(zone1space1)
vav1_to_zone1 = duct_templ.evaluate({"a": BLDG["VAV-1-out"], "b": BLDG["zone1-in"]})
things.append(vav1_to_zone1)

/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "domain-space-sup-flow-sensor, domain-space-exh-flow-sensor, domain-space-out-mapsto, domain-space-supply-air-flow, domain-space-humidity-sensor, domain-space-in, domain-space-temp-sensor, domain-space-exhaust-air-flow, domain-space-out2-mapsto, domain-space-out, domain-space-in2-mapsto, domain-space, domain-space-in4, domain-space-in3-mapsto, domain-space-in4-mapsto, domain-space-in2, domain-space-relative-humidity, domain-space-in-mapsto, domain-space-physical-space, domain-space-temp, domain-space-in3, domain-space-out2" were not provided during evaluation
  warnings.warn(
/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "exhaust-air-flow, sup-flow-sensor, supply-air-flow, in4, out, humidity-sensor, in2-mapsto, relative-humidity, out2-mapsto, out-mapsto, in4-mapsto, in3, in, physical-space, in2, in3-mapsto, temp-sensor, exh-flow-s

In [8]:
# VAV2 goes to FCU1
fcu1 = fcu_templ.evaluate({"name": BLDG["fcu1"]})
vav2_to_fcu1 = duct_templ.evaluate({"a": BLDG["VAV-2-out"], "b": BLDG["fcu1-in"]})
things.extend([fcu1, vav2_to_fcu1])

/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "zone-temp, fan, cooling-coil-pump-out, zone-humidity, cooling-coil-air-out, heating-coil-return-water-temp, heating-coil-air-out, heating-coil-valve-out, fan-vfd-flt, out, cooling-coil, fan-vfd-frq, cooling-coil-pump-vfd-fb, cooling-coil-return-water-temp, cooling-coil-pump-out-mapsto, cooling-coil-pump-vfd-flt, cooling-coil-pump-vfd-frq, fan-motor-status, heating-coil-valve-in, heating-coil-air-out-mapsto, cooling-coil-pump-onoff-cmd, heating-coil-valve, cooling-coil-leaving-air-wetbulb-temp, heating-coil-air-in-mapsto, fan-start-cmd, cooling-coil-valve-out, cooling-coil-pump-vfd-pwr, heating-coil-water-in, cond-overflow, in, cooling-coil-water-in-mapsto, cooling-coil-pump-in, fan-in-mapsto, fan-oa-flow-switch, cooling-coil-valve-command, heating-coil-return-water-temp-sensor, cooling-coil-pump, occ-override, fan-vfd-volt, heating-coil-water-out-mapsto, cooling-coil-air-in-mapsto, co

In [9]:
# FCU1 goes to Zone 2
zone2 = zone_templ.evaluate({"name": BLDG["zone2"]})
zone2space1 = space_templ.evaluate({"name": BLDG["zone2space1"], "zone": BLDG["zone2"]})
fcu1_to_zone2 = duct_templ.evaluate({"a": BLDG["fcu1-out"], "b": BLDG["zone2-in"]})
things.extend([zone2, zone2space1, fcu1_to_zone2])

In [10]:
# add chws, hws
chws = chws_templ.evaluate({"name": BLDG["CHWS"]})
hws = hws_templ.evaluate({"name": BLDG["HWS"]})
things.extend([chws, hws])

/Users/gabe/src/NREL/BuildingMOTIF/buildingmotif/dataclasses/template.py:412: UserWarning: Parameters "lead-chw-booster-pump-vfd-flt, standby-chw-pump-vfd-volt, standby-chw-pump-onoff-cmd, lead-chw-booster-pump-onoff-sts, chw-hx-chw-flow, standby-chw-pump-vfd-spd, lead-chw-pump-in, bypass-valve-in-mapsto, standby-chw-booster-pump, standby-chw-booster-pump-onoff-sts, chw-hx-A-in-mapsto, chw-hx-A-out, lead-chw-pump-vfd-energy, lead-chw-booster-pump, lead-chw-pump-vfd-flt, bypass-valve, bypass-valve-in, chw-hx-chw-flow-sensor, chw-hx-chw-supply-temperature, lead-chw-pump-vfd-fb, lead-chw-booster-pump-vfd-cur, chw-hx-A-chw-diff-press-sensor, chw-hx-chw-return-temperature, lead-chw-booster-pump-in-mapsto, standby-chw-pump-vfd-frq, chw-hx-B-out, standby-chw-booster-pump-vfd-energy, lead-chw-pump-out, lead-chw-pump-vfd-cur, lead-chw-booster-pump-vfd-frq, lead-chw-booster-pump-vfd-spd, lead-chw-booster-pump-vfd-fb, standby-chw-booster-pump-out-mapsto, chw-hx-A-out-mapsto, standby-chw-pump-in-m

In [11]:
# fill in all the extra parameters with invented names
for templ in things:
    if isinstance(templ, Graph):
        bldg.add_graph(templ) # template is already complete!
        continue
    print(templ.name)
    _, graph = templ.fill(BLDG, include_optional = True) # invent URIs for things we don't need names for (like ducts)
    bldg.add_graph(graph)

makeup-air-unit
vav-reheat
duct
vav-reheat
duct
hvac-zone
hvac-space
duct
fcu
duct
hvac-zone
hvac-space
duct
chilled-water-system
hot-water-system


In [12]:
# look at finished model
print(bldg.graph.serialize())
bldg.graph.serialize("output.ttl")

@prefix bldg: <urn:ex/> .
@prefix ns1: <http://data.ashrae.org/standard223#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix qudtqk: <http://qudt.org/vocab/quantitykind/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix unit: <http://qudt.org/vocab/unit/> .

ns1:EnumerationKind-Default a ns1:EnumerationKind-AlarmStatus,
        ns1:EnumerationKind-Override ;
    rdfs:label "AlarmStatus-Alarm"@en,
        "Override-Default"@en .

ns1:EnumerationKind-Overridden a ns1:EnumerationKind-AlarmStatus,
        ns1:EnumerationKind-Override ;
    rdfs:label "AlarmStatus-Ok"@en,
        "Override-Overridden"@en .

bldg: a owl:Ontology .

bldg:CHWS a ns1:System ;
    rdfs:label "Chilled Water System" ;
    ns1:contains bldg:bypass-valve_f8f59d89,
        bldg:chw-hx_8dc037ec,
        bldg:lead-chw-booster-pump_46e000ce,
        bldg:lead-chw-pump_76441788,
        bldg:standby-chw-b

<Graph identifier=2f97ee9a-7e28-44a5-beb6-c0173f4eba28 (<class 'rdflib.graph.Graph'>)>

In [13]:
# validate against 223P
ctx = bldg.validate([s223.get_shape_collection()], error_on_missing_imports=False)
print(ctx.valid)
print(ctx.report_string)

2024-07-02 17:28:44,142 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/vocab/enumeration from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:28:44,146 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/vocab/enumeration from Libraries. Trying shape collections


2024-07-02 17:28:44,147 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.w3.org/ns/shacl# from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:28:44,150 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.w3.org/ns/shacl# from Libraries. Trying shape collections


2024-07-02 17:28:44,151 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/model/core from Libraries (No row was found when one was required). Trying shape collections


...

2024-07-02 17:28:44,165 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/model/equipment from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:28:44,168 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/model/equipment from Libraries. Trying shape collections


2024-07-02 17:28:44,169 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/extensions/settings from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:28:44,172 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/extensions/settings from Libraries. Trying shape collections


0
@prefix ns2: <http://data.ashrae.org/standard223#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <urn:ex/cooling-coil-valve-out-mapsto_baa71c46> ;
            sh:resultMessage "s223: A ConnectionPoint must be associated with exactly one Connectable using the relation isConnectionPointOf." ;
            sh:resultPath ns2:isConnectionPointOf ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinCountConstraintComponent ;
            sh:sourceShape <urn:well-known/5dbd284e> ],
        [ a sh:ValidationResult ;
            sh:focusNode <urn:ex/heating-coil-valve-in-mapsto_7c93b160> ;
            sh:resultMessage "s223: A ConnectionPoint must be associated with exactly one Connectable using the relation isConnectionPointOf." ;
            sh:resultPath ns2:isConnectionPointOf ;
      

In [14]:
bldg.add_graph(mau_templ.fill(BLDG)[1])

In [15]:
ctx = bldg.validate([s223.get_shape_collection()], error_on_missing_imports=False)
print(ctx.valid)
print(ctx.report_string)

2024-07-02 17:29:33,708 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/vocab/enumeration from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:29:33,713 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/vocab/enumeration from Libraries. Trying shape collections


2024-07-02 17:29:33,713 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.w3.org/ns/shacl# from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:29:33,717 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://www.w3.org/ns/shacl# from Libraries. Trying shape collections


2024-07-02 17:29:33,718 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/model/core from Libraries (No row was found when one was required). Trying shape collections


...

2024-07-02 17:29:33,736 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/model/equipment from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:29:33,741 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/model/equipment from Libraries. Trying shape collections


2024-07-02 17:29:33,742 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/extensions/settings from Libraries (No row was found when one was required). Trying shape collections


2024-07-02 17:29:33,745 | buildingmotif.dataclasses.shape_collection |  WARNING: Could not resolve import of http://data.ashrae.org/standard223/1.0/extensions/settings from Libraries. Trying shape collections


0
@prefix ns2: <http://data.ashrae.org/standard223#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <urn:ex/cooling-coil-pump-out-mapsto_21c99d54> ;
            sh:resultMessage "s223: A ConnectionPoint must be associated with exactly one Connectable using the relation isConnectionPointOf." ;
            sh:resultPath ns2:isConnectionPointOf ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinCountConstraintComponent ;
            sh:sourceShape <urn:well-known/4cd76c4c> ],
        [ a sh:ValidationResult ;
            sh:focusNode <urn:ex/evaporative-cooler-evap-cool-fill-valve-in-mapsto_44512a55> ;
            sh:resultMessage "s223: A ConnectionPoint must be associated with exactly one Connectable using the relation isConnectionPointOf." ;
            sh:resultPath ns2:isConnec

In [16]:
print(mau_templ.fill(BLDG)[1].serialize())


@prefix ns1: <http://data.ashrae.org/standard223#> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix qudtqk: <http://qudt.org/vocab/quantitykind/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix unit: <http://qudt.org/vocab/unit/> .

ns1:EnumerationKind-Default a ns1:EnumerationKind-AlarmStatus ;
    rdfs:label "AlarmStatus-Alarm"@en .

ns1:EnumerationKind-Overridden a ns1:EnumerationKind-AlarmStatus ;
    rdfs:label "AlarmStatus-Ok"@en .

<urn:ex/name_124d8021> a ns1:AirHandlingUnit ;
    ns1:contains <urn:ex/MAU-HRC_63b4f623>,
        <urn:ex/cooling-coil_eba92d15>,
        <urn:ex/evaporative-cooler_08f64d72>,
        <urn:ex/final-filter_1ea28bf1>,
        <urn:ex/heating-coil_81095f18>,
        <urn:ex/oad_c88829bc>,
        <urn:ex/pre-filter_6542c32b>,
        <urn:ex/sa_pressure_sensor_b565a126>,
        <urn:ex/supply-fan_62745c4f> ;
    ns1:hasConnectionPoint <urn:ex/air-supply_4d740a0d>,
        <urn:ex/out